# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [7]:
# imports

import os
import json
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [5]:

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key and google_api_key.startswith('AIza'):
    print("Gemini API key looks good so far")
else:
    raise ValueError("Problem with Gemini API key – check your .env")

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

MODEL = "gemini-2.5-flash-lite"  #

Gemini API key looks good so far


In [6]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [12]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [13]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [14]:
DB = "prices.db"

def set_ticket_price(city, price):
    """Insert or update a ticket price in the database"""
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            'INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?',
            (city.lower(), price, price)
        )
        conn.commit()
    print(f"[DB] Updated {city} = ${price}")
    return f"Price for {city} set to ${price}"

In [15]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        fn_name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        print(f"[LLM] Requested tool: {fn_name}, args={args}")

        if fn_name == "get_ticket_price":
            city = args.get("destination_city")
            result = get_ticket_price(city)
        elif fn_name == "set_ticket_price":
            city = args.get("destination_city")
            price = args.get("price")
            result = set_ticket_price(city, price)
        else:
            result = f"Unknown tool: {fn_name}"

        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })

        print(f"[TOOL] Response to LLM: {result}")

    return responses


In [16]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = gemini.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = gemini.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content


In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


[LLM] Requested tool: get_ticket_price, args={'destination_city': 'tokyo'}
DATABASE TOOL CALLED: Getting price for tokyo
[TOOL] Response to LLM: No price data available for this city
[LLM] Requested tool: set_ticket_price, args={'destination_city': 'tokyo', 'price': 400}
[DB] Updated tokyo = $400
[TOOL] Response to LLM: Price for tokyo set to $400
[LLM] Requested tool: set_ticket_price, args={'destination_city': 'Berlin', 'price': 300}
[DB] Updated Berlin = $300
[TOOL] Response to LLM: Price for Berlin set to $300
[LLM] Requested tool: set_ticket_price, args={'destination_city': 'London', 'price': 100}
[DB] Updated London = $100
[TOOL] Response to LLM: Price for London set to $100
[LLM] Requested tool: set_ticket_price, args={'destination_city': 'India', 'price': 200}
[DB] Updated India = $200
[TOOL] Response to LLM: Price for India set to $200
[LLM] Requested tool: get_ticket_price, args={'destination_city': 'london'}
DATABASE TOOL CALLED: Getting price for london
[TOOL] Response to L